# Day 15: Chiton
This has to be a problem for 
[Dijkstra's Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm)
([Computerphile's breakdown](https://www.youtube.com/watch?v=GazC3A4OQTE))
but Dijkstra doesn't account for weights per point it just does shortes path.

A* does account for weights
([Computerphile's breakdown](https://www.youtube.com/watch?v=ySN5Wnu88nE))

In [ ]:
type PathPoint (x:int,y:int,value:int,distance:int) =
    static member (-) (p1:PathPoint, p2:PathPoint) =
        PathPoint (p1.X - p2.X, p1.Y - p2.Y, p1.Value - p1.Value, p1.Distance - p2.Distance)

    static member (+) (p1:PathPoint, p2:PathPoint) =
        PathPoint (p1.X + p2.X, p1.Y + p2.Y, p1.Value + p1.Value, p1.Distance + p2.Distance)

    member this.X = x
    member this.Y = y
    member this.Value = value
    member this.Distance = distance

In [ ]:
let int (ch:char) :int = int ch - int '0'
let array = Seq.map int
let sample = "1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"
let cleanData :int[,] = sample.Split(Environment.NewLine) |> Array.map array |> array2D
let finish :int*int = (cleanData.GetUpperBound(0), cleanData.GetUpperBound(1))
let fth (_,_,_,t) = t

let sub (x1,y1) (x2,y2) = (x1 - x2, y1 - y2)
let square (x,y) = (pown x 2, pown y 2)
/// A truncated version of pythag as we just want a distance modifier
let dist a b = (a,b) ||> sub ||> (+)
let getNeighbors x y = 
    [|
        for a = max (x-1) 0 to min (x+1) (fst finish) do
            for b = max (y-1) 0 to min (y+1) (snd finish) do
                // Exclude ourselves
                if (a = x && b <> y) || (a <> x && b = y) then //  Don't consider diagonals for now
                // if a <> x || b <> y then // Take all neighbors excluding the given point
                    (a,b, cleanData.[a,b], (dist finish (a,b)) + cleanData.[a,b])

    |]

let astar max =
    let mutable kill = 0
    let rec f q =
        //  Pop off the top of our queue
        let h = Array.head q
        h |> display
        //  Decompose h a little for ease of use
        let (x,y,value,_) = h
        //  If our top queue item is our finish, we're done
        if (x = fst finish && y = snd finish) || kill = max then q
        else
            //  Our leftovers for later
            let t = Array.tail q
            kill <- kill + 1
            //  Get the neighbors of H
            getNeighbors x y 
            //  Filter out coords that match where we came from
            // |> Array.filter ()
            //  Boost their values as we progress
            |> Array.map (fun (ax,ay,av,ad) -> (ax,ay,av+value,ad+value))
            //  Append this list to our queue
            |> Array.append t
            //  Sort by distance
            |> Array.sortBy fth
            //  Repeat
            |> f

    getNeighbors 0 0 |> f 

// astar 0

PathPoint (0,0,cleanData.[0,0],dist finish (0,0))

Error: input.fsx (3,1)-(3,5) typecheck error This value is not a function and cannot be applied.
input.fsx (41,9)-(41,21) typecheck warning The result of this expression has type 'DotNet.Interactive.DisplayedValue' and is implicitly ignored. Consider using 'ignore' to discard this value explicitly, e.g. 'expr |> ignore', or 'let' to bind the result to a name, e.g. 'let result = expr'.

In [ ]:
type Config<'a> = 
    {
        /// <summary>
        /// A method that, given a source, will return its neighbours.
        /// </summary>
        neighbours: 'a -> seq<'a>
        /// <summary>
        /// Given two nodes that are next to each other, return the g cost between them.
        /// The g cost is the cost of moving from one to the other directly.
        /// </summary>
        gCost: 'a -> 'a -> float
        /// <summary>
        /// Given two nodes, return the f cost between them. This is a heuristic score used from a given node to the goal.
        /// Line-of-sight distance is an example of how this might be defined.
        /// </summary>
        fCost: 'a -> 'a -> float
        /// <summary>
        /// The maximum number of tiles to check - used to limit overly long searches when accuracy is not paramount
        /// </summary>
        maxIterations: int option
    }

let search<'a when 'a : comparison> start goal config : seq<'a> option =

    let rec reconstructPath cameFrom current =
        seq {
            yield current
            match Map.tryFind current cameFrom with
            | None -> ()
            | Some next -> yield! reconstructPath cameFrom next
        }

    let rec crawler closedSet (openSet, gScores, fScores, cameFrom) =
        match config.maxIterations with 
        | Some n when n = Set.count closedSet -> None
        | _ ->
            match List.sortBy (fun n -> Map.find n fScores) openSet with
            | current::_ when current = goal -> Some <| reconstructPath cameFrom current 
            | current::rest ->
                let gScore = Map.find current gScores
                let next =
                    config.neighbours current 
                    |> Seq.filter (fun n -> closedSet |> Set.contains n |> not)
                    |> Seq.fold (fun (openSet, gScores, fScores, cameFrom) neighbour ->
                        let tentativeGScore = gScore + config.gCost current neighbour
                        if List.contains neighbour openSet && tentativeGScore >= Map.find neighbour gScores 
                        then (openSet, gScores, fScores, cameFrom)
                        else
                            let newOpenSet = if List.contains neighbour openSet then openSet else neighbour::openSet
                            let newGScores = Map.add neighbour tentativeGScore gScores
                            let newFScores = Map.add neighbour (tentativeGScore + config.fCost neighbour goal) fScores
                            let newCameFrom = Map.add neighbour current cameFrom
                            newOpenSet, newGScores, newFScores, newCameFrom
                        ) (rest, gScores, fScores, cameFrom)
                crawler (Set.add current closedSet) next
            | _ -> None

    let gScores = Map.ofList [start, 0.]
    let fScores = Map.ofList [start, config.fCost start goal]
    crawler Set.empty ([start], gScores, fScores, Map.empty)

In [ ]:
let int (ch:char) :int = int ch - int '0'
let sample = "1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"

let grid = sample.Split(Environment.NewLine) |> Array.map (fun s -> Seq.map float s ) //|> array2D
let width,height = Seq.length (Array.head grid), Array.length grid// grid.GetUpperBound(0), grid.GetUpperBound(1)
let start = 0,0
let goal = width - 1, height - 1
let neighbours (x, y) =
        let found = 
            [-1..1] |> List.collect (fun nx ->
            [-1..1] |> List.filter (fun ny -> ny <> nx || ny <> 0) |> List.map (fun ny -> x + nx, y + ny))
        found |> Seq.filter (fun (nx, ny) -> 
            nx > 0 && ny > 0 &&
            nx < width && ny < height)
let g x y = Seq.item x (Array.item y grid)
let fScore (x, y) (gx, gy) = sqrt ((float gx - float x)**2. + (float gy - float y)**2.)

match search start goal { neighbours = neighbours; gCost = g; fCost = fScore; maxIterations = Some(20) } with
| Some path -> display path |> ignore
| None -> printf "No path"

Error: input.fsx (27,60)-(27,61) typecheck error The type 'int * int' does not match the type 'int'